In [20]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import string
import os
from math import *

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
pd.options.display.max_rows = 5

In [5]:
p1 = "/Users/sai/Documents/00 NEU/Semester 1/"
p2 = "1 DS 5110 - Introduction to Data Management and Processing/"
p3 = "Project/NYC-Taxi-Data-Analysis/data/green/"

dpath = p1 + p2 + p3
fname = "green_samp_locid.csv"

In [8]:
green = pd.read_csv(dpath+fname)

In [10]:
slocid = 80
dlocid = 188

In [84]:
subset = (
    green
    .query("(pulocationid == @slocid) & (dolocationid == @dlocid)")
)

In [85]:
subset.head()

,vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pulocationid,dolocationid,trip_distance,...,fare_amount,tip_amount,extra,tolls_amount,improvement_surcharge,ehail_fee,mta_tax,ratecodeid,passenger_count,store_and_fwd_flag
797,2,2015-05-09 23:40:48,2015-05-10 00:06:11,-73.942719,40.714504,-73.955566,40.662827,80.0,188.0,6.30,...,23.0,4.7,0.5,0.0,0.0,NaN,0.5,1,1,N
31054,2,2015-06-27 22:51:15,2015-06-27 23:19:14,-73.948395,40.707222,-73.961487,40.660061,80.0,188.0,5.10,...,21.0,0.0,0.5,0.0,0.3,NaN,0.5,1,1,N
39292,2,2016-08-17 16:47:07,2016-08-17 17:15:00,NaN,NaN,NaN,NaN,80.0,188.0,3.92,...,19.0,2.0,1.0,0.0,0.3,NaN,0.5,1,1,N
50316,2,2016-04-24 21:14:23,2016-04-24 21:35:07,-73.951714,40.714584,-73.953743,40.662563,80.0,188.0,4.06,...,16.5,0.0,0.5,0.0,0.3,NaN,0.5,1,1,N
53309,2,2017-11-14 22:02:46,2017-11-14 22:27:51,NaN,NaN,NaN,NaN,80.0,188.0,6.55,...,23.5,6.2,0.5,0.0,0.3,NaN,0.5,1,3,N


In [86]:
def haversine(slon, slat, dlon, dlat):
    slon, slat, dlon, dlat = map(radians, [slon, slat, dlon, dlat])
    
    dellon = dlon - slon
    dellat = dlat - slat
    
    # Haversine formula
    a = (sin(dellat/2) ** 2) + (cos(slat) * cos(dlat) * sin(dellon/2) ** 2)
    b = 2 * asin(sqrt(a))
    r = 3956
    dist = b * r
    
    return(dist)

In [87]:
subset2 = (
    subset
    .query("pickup_latitude.notnull()")
    .filter(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
             'dropoff_latitude', 'pulocationid', 'dolocationid', 'trip_distance'])
)

In [88]:
subset2['haver_dist'] = subset2.apply(
    lambda row: haversine(row.pickup_longitude, row.pickup_latitude, 
                          row.dropoff_longitude, row.dropoff_latitude), axis=1
)
subset2.assign()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pulocationid,dolocationid,trip_distance,haver_dist
797,-73.942719,40.714504,-73.955566,40.662827,80.0,188.0,6.30,3.630947
31054,-73.948395,40.707222,-73.961487,40.660061,80.0,188.0,5.10,3.327617
...,...,...,...,...,...,...,...,...
7143720,-73.944992,40.718212,-73.957115,40.661903,80.0,188.0,4.84,3.939315
7159942,-73.941879,40.721611,-73.947395,40.661839,80.0,188.0,5.45,4.137096
